### 0 Import Libraries

In [17]:
import networkx as nx
import pandas as pd
import os
from pyvis.network import Network
import json

In [18]:
# Read entities.txt and extract the entities as a list
entities = pd.read_csv('entities.txt', sep='\t', header=None)
entities = entities[0].tolist()

In [19]:
# Read conversion.csv
conversion = pd.read_csv('conversion.csv')

In [20]:
# Use conversion Persian and English columns to map the entities to their English names
# If nothing is found, use the Persian name
entities = [conversion[conversion['Persian'] == entity]['English'].values[0] if entity in conversion['Persian'].values else entity for entity in entities]

In [21]:
# Create a directed graph with the entities as nodes
G = nx.DiGraph()
G.add_nodes_from(entities, color='green')

In [22]:
relationships = pd.read_csv('relationships.txt', sep=',', header=None)

# Convert the relationships to English
relationships = relationships.applymap(lambda x: conversion[conversion['Persian'] == x]['English'].values[0] if x in conversion['Persian'].values else x)
relationships

/var/folders/ps/l9kwl1vj2034rbbqs1vv7ttm0000gn/T/ipykernel_60383/799912092.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  relationships = relationships.applymap(lambda x: conversion[conversion['Persian'] == x]['English'].values[0] if x in conversion['Persian'].values else x)


0              1
0              Amlash   Kalan Kalaye
1    Imamzadeh Hashem          Rasht
2             Saravan          Fuman
3             Lahijan        Astaneh
4          Someh Sara          Fuman
..                ...            ...
325   Jangal Golestan        Tangrah
326  Se-rah-e Kalaleh         Gonbad
327           Aliabad      Azadshahr
328     Khosh Yeylagh      Azadshahr
329         Azadshahr  Khosh Yeylagh

[330 rows x 2 columns]

In [23]:
for i in range(relationships.shape[0]):
    G.add_edge(relationships.iloc[i, 0], relationships.iloc[i, 1], width=1)

In [24]:
# Visualize the graph
nt = Network("800px", "1500px", directed=True, select_menu=True, filter_menu=True, notebook=True)
nt.from_nx(G)

# Read coordinates from the json file
with open('coordinates.json') as f:
    coordinates = json.load(f)

min_lat = 100
min_lng = 100

# Set the coordinates of the nodes
for node in nt.nodes:
    node['size'] = 3
    if node['id'] in coordinates:
        # Find the Province of the node
        if conversion[conversion['English'] == node['id']].shape[0] > 0:
            province = conversion[conversion['English'] == node['id']]['Province'].values[0]
            if province == 'Gilan':
                node['color'] = 'blue'
            elif province == 'Mazandaran':
                node['color'] = 'red'
            elif province == 'Golestan':
                node['color'] = 'green'
            else:
                node['color'] = 'black'
        node['x'] = (coordinates[node['id']]['lng'] - 48) * 500
        node['y'] = -(coordinates[node['id']]['lat'] - 35.5) * 500
    else:
        print(node)
        node['x'] = 0
        node['y'] = 0

        
nt.toggle_physics(False)
nt.show('graph.html', notebook=False)
# Open the html file in the browser
os.system('open graph.html')

{'color': 'green', 'size': 3, 'id': 'Golestan', 'label': 'Golestan', 'shape': 'dot'}
{'color': 'green', 'size': 3, 'id': 'جنوبي بابل', 'label': 'جنوبي بابل', 'shape': 'dot'}
{'color': 'green', 'size': 3, 'id': 'شمال', 'label': 'شمال', 'shape': 'dot'}
{'color': 'green', 'size': 3, 'id': 'Mehvar Tuskestan', 'label': 'Mehvar Tuskestan', 'shape': 'dot'}
{'color': 'green', 'size': 3, 'id': 'متناظر سامانه توزين', 'label': 'متناظر سامانه توزين', 'shape': 'dot'}
graph.html


0

In [10]:
conversion

Persian            English
0             فومن              Fuman
1            جيرده             Jirdeh
2           کياشهر           Kiashahr
3           چابکسر          Chaboksar
4              رشت              Rasht
..             ...                ...
130  سه راهي کلاله   Se-rah-e Kalaleh
131   تقاطع سراوان  Taghato-e Saravan
132  سه راهي يساقي             Yasaqi
133  سه راه سنگچال          Sang Chal
134      مرز استان       Rasht Border

[135 rows x 2 columns]